In [ ]:
import ifc_utils
import keras
import numpy
import matplotlib.pyplot
import os.path
import pandas

import keras.applications
import keras.preprocessing.image
import tensorflow

# User's settings

In [ ]:
directory = "/home/minh-doan/Cell_cycle/temp_processed_Juan"

data = {
    #"G1": "/home/minh-doan/Cell_cycle/Step2_input_single_tifs/G1/",
    #"S_phase": "/home/minh-doan/Cell_cycle/Step2_input_single_tifs/S_phase/",
    #"G2": "/home/minh-doan/Cell_cycle/Step2_input_single_tifs/G2/",
    #"Prophase": "/home/minh-doan/Cell_cycle/Step2_input_single_tifs/Prophase/",
    "Anaphase": "/home/minh-doan/Cell_cycle/Step2_input_single_tifs/Anaphase/",
    "Metaphase": "/home/minh-doan/Cell_cycle/Step2_input_single_tifs/Metaphase/",
    "Telophase": "/home/minh-doan/Cell_cycle/Step2_input_single_tifs/Telophase/"
}

channels = [3]

image_size = 40

split = {
    "Training" : 0.8,
    "Validation" : 0.1,
    "Testing" : 0.1
}

In [ ]:
class_weights = ifc_utils.parse(directory, data, channels, image_size, split)

In [ ]:
shape = (image_size, image_size, 1)

x = keras.layers.Input(shape)

classes = len(data.keys())

The classifier architecture is adapted from the LeNet-5 convolutional neural network by Yann LeCunn, et al.

In [ ]:
options = {"activation": "relu", "kernel_size": (3, 3)}

y = keras.layers.Conv2D(32, **options)(x)
y = keras.layers.Conv2D(64, **options)(y)

y = keras.layers.MaxPooling2D()(y)

y = keras.layers.Dropout(0.5)(y)

y = keras.layers.Flatten()(y)

y = keras.layers.Dense(128, activation="relu")(y)

y = keras.layers.Dropout(0.5)(y)

y = keras.layers.Dense(classes)(y)

y = keras.layers.Activation("softmax")(y)

In [ ]:
model = keras.models.Model(x, y)

In [ ]:
loss = keras.losses.categorical_crossentropy

optimizer = keras.optimizers.Adam(0.0001)

model.compile(
    loss=loss, 
    metrics=[
        "accuracy"
    ],
    optimizer=optimizer
)

In [ ]:
training_x = numpy.load(os.path.join(directory, "training_x.npy"))

training_y = numpy.load(os.path.join(directory, "training_y.npy"))

In [ ]:
training_generator = keras.preprocessing.image.ImageDataGenerator(rotation_range = 180, horizontal_flip = True, vertical_flip = True)

training_generator = training_generator.flow(
    x = training_x,
    y = training_y,
    batch_size=32
)

In [ ]:
validation_x = numpy.load(os.path.join(directory, "validation_x.npy"))

validation_y = numpy.load(os.path.join(directory, "validation_y.npy"))

In [ ]:
validation_generator = keras.preprocessing.image.ImageDataGenerator(rotation_range = 180, horizontal_flip = True, vertical_flip = True)

validation_generator = validation_generator.flow(
    x = validation_x,
    y = validation_y,
    batch_size=32
)

In [ ]:
csv_logger = keras.callbacks.CSVLogger(os.path.join(directory, 'training.csv') )

early_stopping = keras.callbacks.EarlyStopping(patience=64)

# checkpoint
filepath = os.path.join(directory, "weights.best.hdf5")
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
configuration = tensorflow.ConfigProto()

configuration.gpu_options.allow_growth = True

session = tensorflow.Session(config=configuration)

keras.backend.set_session(session)

In [ ]:
with tensorflow.device("/gpu:2"):
    model.fit_generator(
        callbacks=[
            checkpoint
            #csv_logger
        ],
        epochs=1,
        class_weight= class_weights,
        generator=training_generator,
        max_q_size=256,
        steps_per_epoch=2000,
        validation_data=validation_generator,
        validation_steps=2000
    )

In [ ]:
testing_x = numpy.load(os.path.join(directory, "testing_x.npy"))

testing_y = numpy.load(os.path.join(directory, "testing_y.npy"))

In [ ]:
test_generator = keras.preprocessing.image.ImageDataGenerator(rotation_range = 180, horizontal_flip = True, vertical_flip = True)

test_generator = test_generator.flow(
    x = testing_x,
    y = testing_y,
    batch_size=32
)

model.evaluate_generator(
    generator=test_generator, 
    steps=256
)

In [ ]:
session.close()